In [9]:
import numpy as np
import pandas as pd
import pickle as pk
from matplotlib import  pyplot as plt
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split # Split the datasets into training and testing
from sklearn.metrics import r2_score,mean_squared_error# Our cost function for simple linear regression

Will read the data using pandas 

In [20]:
Gold_Price = pd.read_csv("gld_price_data.csv")
Gold_Price.head()
Gold_Price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     2290 non-null   object 
 1   SPX      2290 non-null   float64
 2   GLD      2290 non-null   float64
 3   USO      2290 non-null   float64
 4   SLV      2290 non-null   float64
 5   EUR/USD  2290 non-null   float64
dtypes: float64(5), object(1)
memory usage: 107.5+ KB


let's check if there are nulls

In [11]:
Gold_Price.isnull().sum()

Date       0
SPX        0
GLD        0
USO        0
SLV        0
EUR/USD    0
dtype: int64

No Nulls and that is good, hope all the datasets are like these (LoL)
Let's check the correlation

In [21]:
Gold_Price = Gold_Price.drop('Date',axis=1)
corr_vals = Gold_Price.corr()['EUR/USD'].drop('EUR/USD')
corr_vals
Gold_Price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SPX      2290 non-null   float64
 1   GLD      2290 non-null   float64
 2   USO      2290 non-null   float64
 3   SLV      2290 non-null   float64
 4   EUR/USD  2290 non-null   float64
dtypes: float64(5)
memory usage: 89.6 KB


As shown GLD feature has small effect on the price of gold 

In [13]:

#scaler = StandardScaler()
features = Gold_Price.iloc[:,[0,1,2,3]]
target = Gold_Price.iloc[:,4]
#X = scaler.fit_transform(features)


In [14]:
# Train Test Split
X_train,X_test,y_train,y_test = train_test_split(features,target,random_state=42,test_size=0.2)

# Create and train the model (simple linear reg and XGboost) --> it would be a better solution if we used pipelining
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

XGRegModel = XGBRegressor()
XGRegModel.fit(X_train,y_train) 


# Make Predictions and evaluate
y_predict_linear = linear_model.predict(X_test)
y_predict_XGBoost = linear_model.predict(X_test)

# Linear Model Performance
R2_score_linear = r2_score(y_test,y_predict_linear)
MSE_metric_linear = mean_squared_error(y_test,y_predict_linear)

# XGBoosting Model Performance
R2_score_XG = r2_score(y_test,y_predict_XGBoost)
MSE_metric_XG = mean_squared_error(y_test,y_predict_XGBoost)



R2 score of linear model : 0.775 and R2 score of XGBoost REgressor : 0.774
Seems like no more improvement we can do

In [15]:
# Input Data for model predictions
input_data = (1400.16,80.86,90.57,17.18)
input_data = np.asarray(input_data)
prediction = XGRegModel.predict(input_data.reshape(1,-1))

prediction[0]

np.float32(1.5712526)

Saving the trained model

In [18]:
filename = 'traine_model.sav'
pk.dump(XGRegModel,open(filename,'wb')) # writing the model into filename file wb ---> write binary


Loading the saved model

In [19]:
laoded_model = pk.load(open('model.sav','rb')) # load the model and read it in binary format